In [15]:
import numpy as np
import torch
import tree
import yaml
import ml_collections

In [2]:
torch.cuda.is_available()

True

In [6]:
import physicsnemo.sym

In [10]:
import sys
sys.path.append('../../')

# from deeponet.mio_dataset import preprocess
from deeponet.data.dataset import MioDataset, preprocess
from deeponet.modulus.modules import MioBranchNet

In [25]:
DATA_PATH = "/workspaces/deeprte/data/raw/train/no-scattering-sigma_a3-sigma_t6/no-scattering-sigma_a3-sigma_t6.npz"
BRANCH_KEYS = ['sigma_a', 'boundary']
TRUNK_KEYS = ['phase_coords']
LABEL_KEY = 'psi_label'

np_data = dict(np.load(DATA_PATH, allow_pickle=True))

In [26]:
cfg_path = "/workspaces/deeprte/deeponet/config/J.yaml"

with open(cfg_path, "r") as f:
    cfg = yaml.safe_load(f)
    cfg = ml_collections.ConfigDict(cfg)

In [27]:
branch_dr, trunk_dr, label_dr, input_shape_dict = preprocess(cfg, np_data)

In [28]:
# dataset = MioDataset(np_data, BRANCH_KEYS, TRUNK_KEYS, 'psi_label', collocation_size=20)
dataset = MioDataset(branch_dr, trunk_dr, label_dr, collocation_size=20)
tree.map_structure(lambda x: x.shape, next(dataset))

{'sigma_a': torch.Size([1600]),
 'boundary': torch.Size([1920]),
 'phase_coords': torch.Size([20, 4]),
 'psi_label': torch.Size([20])}

In [40]:
dataloaders = torch.utils.data.DataLoader(dataset, batch_size=2, num_workers=0)
inputs = next(iter(dataloaders))

In [41]:
inputs.keys()

dict_keys(['sigma_a', 'boundary', 'phase_coords', 'psi_label'])

In [45]:
from functools import reduce
# inputs = reduce(lambda x, y: {**x, **y}, inputs)
tree.map_structure(lambda x: x.shape, inputs)

{'sigma_a': torch.Size([2, 1600]),
 'boundary': torch.Size([2, 1920]),
 'phase_coords': torch.Size([2, 20, 4]),
 'psi_label': torch.Size([2, 20])}

In [ ]:
from physicsnemo.sym.models.fully_connected import FullyConnectedArch
from physicsnemo.sym.models.fourier_net import FourierNetArch
from physicsnemo.sym.models.deeponet import DeepONetArch
from physicsnemo.sym.key import Key
import torch

arch_1 = FullyConnectedArch(
    [Key("sigma_a", size=3200)], [Key("b1", size=128)], layer_size=128, nr_layers=2
)
arch_2 = FullyConnectedArch(
    [Key("boundary", size=1920)], [Key("b2", size=128)], layer_size=128, nr_layers=2
)
branch_net = MioBranchNet([arch_1, arch_2], [Key("branch", size=128)])
# arch_3 = FullyConnectedArch(
#     [Key("x3", size=2)], [Key("b3", size=2)], layer_size=64, nr_layers=2
# )

trunk_net = FourierNetArch(
        input_keys=[Key("phase_coords", size=4)],
        output_keys=[Key("trunk", 128)],
    )

deeponet = DeepONetArch(
        output_keys=[Key("u")],
        branch_net=branch_net,
        trunk_net=trunk_net,
    )

model = deeponet.make_node("deeponet")
# model = branch_net.make_node("branch_net")
# inputs = {"sigma_a": torch.randn(16, 3200), "boundary": torch.randn(16, 1920)}
# # inputs = {k: torch.from_numpy(v) for k, v in inputs.items()}
# inputs.update({"x": torch.randn(16, 2), 'phase_coords': torch.randn(16, 4, 4)})
# output = model.evaluate(inputs)
output= model(inputs)

RuntimeError: Tensors must have same number of dimensions: got 2 and 3

In [6]:
output

{'u': tensor([[ 9.1977e-05],
         [-1.8999e-05],
         [-1.2630e-04],
         [ 9.6460e-05],
         [-5.8578e-05],
         [ 2.8761e-04],
         [ 2.4340e-05],
         [ 6.7943e-06],
         [-2.2871e-04],
         [-1.6994e-04],
         [-2.5038e-05],
         [ 2.6192e-05],
         [ 7.3439e-05],
         [-4.8227e-06],
         [-1.5742e-05],
         [ 7.2662e-05]], grad_fn=<SplitWithSizesBackward0>)}

In [7]:
print(deeponet)

DeepONetArch(
  (branch_net): MioBranchNet(
    (0): FullyConnectedArch(
      (_impl): FullyConnectedArchCore(
        (layers): ModuleList(
          (0): FCLayer(
            (linear): WeightNormLinear(in_features=3200, out_features=128, bias=True)
          )
          (1): FCLayer(
            (linear): WeightNormLinear(in_features=128, out_features=128, bias=True)
          )
        )
        (final_layer): FCLayer(
          (linear): Linear(in_features=128, out_features=128, bias=True)
        )
      )
    )
    (1): FullyConnectedArch(
      (_impl): FullyConnectedArchCore(
        (layers): ModuleList(
          (0): FCLayer(
            (linear): WeightNormLinear(in_features=1920, out_features=128, bias=True)
          )
          (1): FCLayer(
            (linear): WeightNormLinear(in_features=128, out_features=128, bias=True)
          )
        )
        (final_layer): FCLayer(
          (linear): Linear(in_features=128, out_features=128, bias=True)
        )
      )
  

In [13]:
list_1 = [torch.randn(16, 3200), torch.randn(16, 3200), torch.randn(16, 3200)]

In [22]:
torch.randn(16, 3200).view(-1).shape

torch.Size([51200])